In [2]:
#pip install flask skyfield

In [ ]:
from flask import Flask, render_template, jsonify
from datetime import datetime, timedelta
from skyfield.api import load, EarthSatellite, Topos
import numpy as np
import random
import threading
import time

app = Flask(__name__)

# Constants
ground_stations = {
    "University of South Australia": Topos(latitude_degrees=-34.8098, longitude_degrees=138.6200),
    "Canberra Deep Space Communications Complex": Topos(latitude_degrees=-35.40139, longitude_degrees=148.98167),
    "Deakin University": Topos(latitude_degrees=-37.847303, longitude_degrees=145.115831),
}

# Simulated satellite data
satellite_data = {
    "LEO": [
        ("STARLINK-3996", "1 52617U 22052V   24235.84331967 -.00000281  00000+0  46458-6 0  9993"),
        ("STARLINK-3983", "1 52618U 22052W   24235.84699805  .00000907  00000+0  75080-4 0  9997"),
        ("STARLINK-3741", "1 52619U 22052X   24235.83963931  .00002415  00000+0  16977-3 0  9996"),
    ],
}

# Load satellites
def load_satellites():
    satellites = {}
    for category, sat_data in satellite_data.items():
        satellites[category] = [
            EarthSatellite(line, '', name, load.timescale()) for name, line in sat_data
        ]
    return satellites

satellites = load_satellites()

# Background simulation
def run_simulation():
    global current_data
    ts = load.timescale()
    current_data = {
        "satellite_positions": [],
        "throughput": []
    }

    while True:
        now = ts.now() + timedelta(seconds=random.randint(0, 3600))
        sat = random.choice(satellites["LEO"])
        position = sat.at(now).position.au
        throughput = random.randint(1, 10) * 1e6  # Simulated throughput in bps

        current_data["satellite_positions"].append(position)
        current_data["throughput"].append(throughput)

        time.sleep(1)

# API to get simulation data
@app.route('/api/data')
def get_data():
    return jsonify(current_data)

# Serve the main page
@app.route('/')
def index():
    return render_template('index.html')

if __name__ == "__main__":
    threading.Thread(target=run_simulation, daemon=True).start()
    app.run(debug=False, load_dotenv=False, port=5002)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5002/ (Press CTRL+C to quit)


In [ ]:
from flask import Flask, jsonify
import numpy as np
from datetime import datetime
from skyfield.api import load, EarthSatellite, Topos
import random

app = Flask(__name__)

# Load satellite data and initialize timescale
ts = load.timescale()
tle_data = {
    "LEO": [
        ("STARLINK-3996", "1 52617U 22052V   24235.84331967 -.00000281  00000+0  46458-6 0  9993",
         "2 52617  53.2167 280.3292 0001521  86.0935 274.0231 15.08844460125864"),
        # More LEO satellites
    ],
    "GEO": [
        ("INMARSAT 4-F1", "1 28628U 05009A   24235.80508400  .00000026  00000+0  00000+0 0  9997",
         "2 28628   4.3477  41.3195 0002043 142.3871 255.7987  1.00272317 71027"),
        # More GEO satellites
    ]
}

def load_satellites():
    satellites = {"LEO": [], "GEO": []}
    for category, sats in tle_data.items():
        satellites[category] = [
            EarthSatellite(line1, line2, name, ts) for name, line1, line2 in sats
        ]
    return satellites

satellites = load_satellites()
ground_stations = {
    "Station 1": Topos(latitude_degrees=-34.8098, longitude_degrees=138.6200),
    # Add more ground stations
}

@app.route('/api/data')
def api_data():
    now = ts.now()
    satellite_positions = []

    # Get current positions for satellites
    for category, sats in satellites.items():
        for sat in sats:
            pos = sat.at(now).position.km  # Position in kilometers
            satellite_positions.append({
                "name": sat.name,
                "type": category,
                "position": pos.tolist()
            })

    # Simulate throughput
    throughput = [random.randint(1, 10) * 1e6 for _ in range(len(satellite_positions))]

    # Create JSON response
    data = {
        "satellite_positions": satellite_positions,
        "throughput": throughput
    }
    return jsonify(data)

if __name__ == '__main__':
    app.run(debug=False, load_dotenv=False, port=5002)